In [ ]:
# Testing Cell
import inspect

import aviary.api as av
from aviary.subsystems.aerodynamics.aerodynamics_builder import CoreAerodynamicsBuilder
from aviary.utils.doctape import get_variable_name, glue_variable

current_glued_vars = []

CoreAerodynamicsBuilder
glue_variable(get_variable_name(CoreAerodynamicsBuilder), md_code=True)

HEIGHT_ENERGY = av.EquationsOfMotion.HEIGHT_ENERGY
glue_variable('height_energy', HEIGHT_ENERGY.value, md_code=False)

TWO_DEGREES_OF_FREEDOM = av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM
glue_variable(TWO_DEGREES_OF_FREEDOM.value, md_code=False)

# glue all arguments of function CoreAerodynamicsBuilder.__init__()
sigs = inspect.signature(CoreAerodynamicsBuilder)
parameters = sigs.parameters
for name, param in parameters.items():
    glue_variable(name, md_code=True)
    # print(f'Name: {name}, Default: {param.default}, Kind: {param.kind}')

<!-- Work in progress... -->
# Aerodynamics Subsystem

The built-in aerodynamics subsystem in Aviary offers multiple options for computing drag. Users can select from methods based on the FLOPS or GASP legacy codes. Choice of which legacy code's routines to use is determined by the {glue:md}`code_origin` option provided when initializing a {glue:md}`CoreAerodynamicsBuilder`. When using Aviary's [Level 1 interface](../getting_started/onboarding_level1), the code origin for aerodynamics is automatically set to match with the mission method ({glue:md}`height_energy` is paired with FLOPS, and {glue:md}`2DOF` is paired with GASP). Future updates to Aviary will allow for the user to specify aerodynamics code origin directly in the input file.

In [ ]:
from aviary.api import CoreAerodynamicsBuilder, LegacyCode

aero_builder = CoreAerodynamicsBuilder(name='aero_example', code_origin=LegacyCode.FLOPS)

In [ ]:
# Testing Cell
from aviary.interface.methods_for_level1 import run_aviary
from aviary.utils.doctape import glue_class_functions

# Get all functions of class CoreAerodynamicsBuilder
glue_class_functions(
    CoreAerodynamicsBuilder, current_glued_vars, prefix=CoreAerodynamicsBuilder.__name__
)

# glue all argument of function run_aviary()
sigs = inspect.signature(run_aviary)
parameters = sigs.parameters
for name, param in parameters.items():
    glue_variable(name, md_code=True)

Both FLOPS and GASP methods have only a single option for pre-mission components, so there are no user-configurable options when calling {glue:md}`CoreAerodynamicsBuilder.build_pre_mission()`.

For mission analysis, a variety of methods are available to both legacy codes, each with unique options. This can be configured per-mission segment in a {glue:md}`phase_info` file, with relevant aerodynamics configurations placed inside `<phase_name>['core_subsystems']`. The following example would instruct Aviary to send the defined options to a subsystem named 'aero_example' when building the mission, if the dictionary is properly added to {glue:md}`phase_info`

In [ ]:
# Testing Cell
from aviary.api import CoreAerodynamicsBuilder
from aviary.utils.doctape import check_args

check_args(CoreAerodynamicsBuilder.build_pre_mission, ['aviary_inputs', 'kwargs'])

In [ ]:
from aviary.api import AviaryValues

# If you are using Aviary's interface to run analysis, this example dictionary must be
# place in phase_info, under the phase you want the arguments applied, like this:
# (<phase_name>[<subsystem_name>] = aerodynamic_args).

# The aerodynamics subsystem name must match what you provide here
aerodynamic_args = {'aero_example': {'method': 'computed', 'gamma': 1.35}}

# If you are manually building your subsystems, you can instead directly pass the
# arguments to the builder
input_variables = AviaryValues()  # include your aircraft inputs here
mission_comp = aero_builder.build_mission(
    num_nodes=1, aviary_inputs=input_variables, kwargs=aerodynamic_args['aero_example']
)

## FLOPS Based

The choice of using FLOPS based aerodynamics behavior is user specified per mission. Note, detailed wing weight input variables do not impact aerodynamic calculations.

The FLOPS aerodynamics pre-mission component performs calculations to determine the design Mach and lift coefficient of the aircraft.

<!--TODO are these variables actually required or will defaults work?-->
The following input variables are required for the pre-mission calculations:
- {glue:md}`Aircraft.Wing.MAX_CAMBER_AT_70_SEMISPAN` : Maximum camber at 70 percent semi-span, percent of local chord
- {glue:md}`Aircraft.Design.BASE_AREA` : Aircraft base area (total exit cross-section area minus inlet capture areas for internally mounted engines)
- {glue:md}`Aircraft.Wing.AIRFOIL_TECHNOLOGY` : Airfoil technology parameter. Limiting values are: 1.0 represents conventional technology wing (Default); 2.0 represents advanced technology wing
<!--TODO: etc. is there a better way than manual copy/paste?-->

In [ ]:
# Testing Cell
from aviary.api import Aircraft

glue_variable(get_variable_name(Aircraft.Wing.MAX_CAMBER_AT_70_SEMISPAN), md_code=True)
glue_variable(get_variable_name(Aircraft.Wing.AIRFOIL_TECHNOLOGY), md_code=True)
glue_variable(get_variable_name(Aircraft.Design.BASE_AREA), md_code=True)

For mission analysis, FLOPS-based aerodynamics has several choices of method to determine the total lift and drag on the vehicle, listed below. The default is `computed`. If another method is desired, it should be specified in {glue:md}`phase_info` for each individual mission segment.
- `computed`: uses regression-based techniques to estimate lift and drag
- `low_speed`: for use in detailed takeoff analysis, and includes high-lift devices and considers angle-of-attack
- `tabular`: allows the user to substitute the lift and drag coefficient calculations in `computed` with data tables
- `external`: disables Aviary's core aerodynamics computation, intended for use with external subsystems to replace all aerodynamic calculations.

### Computed Aerodynamics
The FLOPS based aerodynamics subsystem uses a modified version of algorithms from the EDET (Empirical Drag Estimation Technique) program [^edet] to internally compute drag polars. FLOPS improvements to EDET as implemented in Aviary include smoothing of drag polars, more accurate Reynolds number calculations, and use of the Sommer and Short T' method [^tprime] for skin friction calculations.

### Low Speed Aerodynamics
This aerodynamics routine is designed for use with the {glue:md}`height_energy` detailed takeoff phase, which includes use of high-lift devices. This aerodynamics method uses angle of attack, which is a special case not present in other {glue:md}`height_energy` phases.

### User Specified Tabular Drag Polars
Third party drag polars can be specified by the user, either via a data file in [Aviary data format](./input_files) or a NamedValues object. Two tables are required, one for lift-dependent drag, and another for zero-lift drag.
- The lift-dependent drag coefficient table must include Mach number and lift coefficient as independent variables.
- The zero-lift drag coefficient table must include altitude and Mach number as independent variables.

Tabular aerodynamics uses Aviary's [data_interpolator_builder](../_srcdocs/packages/utils/data_interpolator_builder) interface. This component is unique as it requires two data tables to be provided. All configuration options, such as the choice to use a structured metamodel or training data, are applied to both tables.

### External Aerodynamics
Selecting the `external` aerodynamics method disables Aviary's core aerodynamics group. This allows for external subsystems to completely replace these calculations.

In [ ]:
# Testing Cell
from aviary.api import AviaryValues, CoreAerodynamicsBuilder, LegacyCode

cab = CoreAerodynamicsBuilder(code_origin=LegacyCode.FLOPS)
# here we are only checking that the CoreAerodynamicsBuilder has a build_mission for a given method
# we know this will fail when it attempts to build the aero groups
for method in (None, 'computed', 'low_speed', 'tabular', 'external'):
    try:
        cab.build_mission(1, AviaryValues(), method=method)
    except ValueError as e:
        raise e
    except:
        pass
try:
    cab.build_mission(1, AviaryValues(), method='computed', solve_alpha=True)
except KeyError:
    pass

## GASP Based
### Using GASP Aerodynamics with the Height-Energy Equations of Motion (FLOPS Mission)

You can also use GASP-based aero with the {glue:md}`height_energy` mission by using the `solve_alpha` flag.

GASP-based drag polars have 3 inputs: altitude, Mach number, and angle of attack. Since the {glue:md}`height_energy` equations of motion do not incorporate angle of attack, `solve_alpha` adds a computational group with a solver that varies the angle of attack until the lift from the FLOPS aero module matches the weight force on the aircraft. When using tabular aerodynamics, the format for the drag table file is the same as for GASP-based aerodynamics used with the {glue:md}`2DOF` mission.

## Externally Computed Polars

Both FLOPS and GASP methods that use data tables support the use of training data, where the values for interpolation are provided by another openMDAO component via connections. An example problem using this method can be found [here](./external_aero).

## References

These papers provide a thorough overview of the specific methodologies used for emperical drag prediction in Aviary.

[^edet]: Feagin, Richard C. and Morrison, William D., Jr. "Delta Method, An Empirical
Drag Buildup Technique." NASA CR-151971, December 1978.

[^tprime]: Sommer, Simon C. and Short, Barbara J. "Free-Flight Measurements of
Turbulent-Boundary-Layer Skin Friction in the Presence of Severe Aerodynamic Heating at
Mach Numbers from 2.8 to 7.9." NASA TN-3391, 1955.